# Closed loop position control of a small DC motor

In [ ]:
using OMJulia, Plots, Interact, WebIO, DataFrames

## Discrete proportional controller

In [ ]:
dcmotor_m = OMJulia.OMCSession()
sendExpression(dcmotor_m, "loadModel(Modelica)")
sendExpression(dcmotor_m, "loadFile(\"../OpenBLDC/package.mo\")")
ModelicaSystem(dcmotor_m, "../OpenBLDC/package.mo", "OpenBLDC.Tests.ClosedLoop.DCMotorPositionDiscrete", ["Modelica"])
setSimulationOptions(dcmotor_m, ["stopTime=1.0", "tolerance=1e-8"])
gr(size=(600,300))

In [ ]:
function sim_DCMotorPositionController(dcmotor_m, parameters)
    k_p, t_s = parameters
    setParameters(dcmotor_m, ["k_p=$k_p", "t_s=$t_s"])
    simulate(dcmotor_m)
    sol = getSolutions(dcmotor_m, ["time", "positionController.y"])
    time, y = sol[1], sol[2]
    plt = plot(time, y, label="", xlabel="time [s]", ylabel="y [-]")
    return plt
end

sim_DCMotorPositionController(dcmotor_m, (25.0, 0.02))

In [ ]:
function test_DCMotorPositionController(dcmotor_m, k_p_vals, t_s_vals)
    k_p_slider = slider(k_p_vals, label = "k_p")
    t_s_slider = slider(t_s_vals, label = "t_s")
    update = button("Update plot")
    parameters = Interact.@map (&update; (k_p_slider[], t_s_slider[]))
    # print(parameters)
    plt = Interact.@map sim_DCMotorPositionController(dcmotor_m, &parameters)
    wdg = Widget(["k_p_slider" => k_p_slider, "t_s_slider" => t_s_slider, "update" => update])
    @layout! wdg hbox(plt, vbox(:k_p_slider, :t_s_slider, :update))
end

In [ ]:
ui = test_DCMotorPositionController(dcmotor_m, 0.0:5.0:100.0, 0.005:0.0025:0.1)
# display(ui)  # Currently not working with WSL2 because of 404 GET /assetserver/...

In [ ]:
using Blink
w = Window()
body!(w, ui);

## A simpler approach using `@manipulate`

In [ ]:
using OMJulia, Plots, Interact, WebIO, DataFrames

dcmotor_m = OMJulia.OMCSession()
sendExpression(dcmotor_m, "loadModel(Modelica)")
sendExpression(dcmotor_m, "loadFile(\"../OpenBLDC/package.mo\")")
ModelicaSystem(dcmotor_m, "../OpenBLDC/package.mo", "OpenBLDC.Tests.ClosedLoop.DCMotorPositionDiscrete", ["Modelica"])
setSimulationOptions(dcmotor_m, ["stopTime=1.0", "tolerance=1e-8"])
gr(size=(700,300))

ui = @manipulate for k_p = 0.0:5.0:100.0, t_s = 0.005:0.0025:0.1
  setParameters(dcmotor_m, ["k_p=$k_p", "t_s=$t_s"])
  simulate(dcmotor_m);
  sol = getSolutions(dcmotor_m, ["time", "positionController.y"])
  time, y = sol[1], sol[2]
  p1 = plot(time, y, label="", xlabel="time [s]", ylabel="y [-]")
  plot(p1)
end

In [ ]:
# display(ui)  # Currently not working with WSL2 because of 404 GET /assetserver/...

In [ ]:
using Blink
w = Window()
body!(w, ui);

## Controller "Software in the loop" using ModelicaDeviceDrivers

In [ ]:
using OMJulia, Plots, Interact, WebIO, DataFrames

controller_exefile = `../OpenBLDC/Resources/build/test/Communication/TCPIPClientAsClosedLoopPositionController`;
controller_task = @task begin; run(controller_exefile); println("done"); end
dcmotor_m = OMJulia.OMCSession()
sendExpression(dcmotor_m, "loadModel(Modelica)")
sendExpression(dcmotor_m, "loadFile(\"../OpenBLDC/package.mo\")")
ModelicaSystem(dcmotor_m, "../OpenBLDC/package.mo", "OpenBLDC.Tests.ClosedLoop.DCMotorPositionTcpIP", ["Modelica"])
setSimulationOptions(dcmotor_m, ["stopTime=1.0", "tolerance=1e-8"])
gr(size=(700,300))

In [ ]:
# Not working yet...

ui = @manipulate for k_p = 0.0:5.0:100.0, t_s = 0.005:0.0025:0.1
    setParameters(dcmotor_m, ["k_p=$k_p", "t_s=$t_s"])
    controller_task = @task begin; sleep(3); println("start controller"); run(controller_exefile); println("controller done"); end
    om_task = @task begin; simulate(dcmotor_m); println("sim done"); end
    schedule(om_task); schedule(controller_task); wait(om_task)
    #simulate(dcmotor_m);
    sol = getSolutions(dcmotor_m, ["time", "positionController.y"])
    time, y = sol[1], sol[2]
    p1 = plot(time, y, label="", xlabel="time [s]", ylabel="y [-]")
    plot(p1)
end

In [ ]:
controller_exefile = `../OpenBLDC/Resources/build/test/Communication/TCPIPClientAsClosedLoopPositionController`;
controller_task = @task begin; run(controller_exefile); println("done"); end

In [ ]:
schedule(controller_task);

In [ ]:
mycommand = `echo hello`

In [ ]:
typeof(mycommand)

In [ ]:
run(mycommand);